Since, other notebooks in this assignment have already discussed the EDA and Linear Regression Model based on OLS method, 

I attempted to introduce Decision Trees, cross-validation on decision trees and random forests to compare the predictive power of each learning algorithms on the given dataset.

In the conclusion, i have concluded with a table prediciting top 5 features for "Males" and "Females" that are significant in predicitng charges. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'): 
    for filename in filenames:
        print(os.path.join(dirname,filename))

In [ ]:


import numpy as np
import pandas as pd


In [ ]:
## reading the insurance dataset

insurance=pd.read_csv("/kaggle/input/insurance/insurance.csv")

In [ ]:
insurance.info()

In [ ]:
insurance.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Predict the dataset on basis of males and females

In [ ]:
## checking the null percentage in the columns of the dataset

(insurance.isnull().sum()/len(insurance.index))*100

In [ ]:
insurance["smoker"]=insurance["smoker"].apply(lambda x: 1 if x=="yes" else 0)

In [ ]:
## Creating males and females dataset for predicting charges as per the gender

male_cost=insurance[insurance["sex"]=="male"]
female_cost=insurance[insurance["sex"]=="female"]

In [ ]:
male_cost.head()

In [ ]:
female_cost.head()

In [ ]:
males=male_cost.drop(["sex"], axis=1)
females=female_cost.drop(["sex"], axis=1)

In [ ]:
male_cost.head()

In [ ]:
males.head()

In [ ]:
females.head()

In [ ]:
males.age.describe()

In [ ]:
males["age"]=males["age"].apply(lambda x: "young_adulthood" if x>=18 and x<=35 else "middle_aged" if x>35 and x<=45 else "older_adulthood")

In [ ]:
males["age"].value_counts(normalize=True)

In [ ]:
females["age"]=females["age"].apply(lambda x: "young_adulthood" if x>=18 and x<=35 else "middle_aged" if x>35 and x<=45 else "older_adulthood")

In [ ]:
females["age"].value_counts(normalize=True)

In [ ]:
males.head()

In [ ]:
males["region"].value_counts(normalize=True)

In [ ]:
### Creating dummies for categorical variables

In [ ]:
dummy_males = pd.get_dummies(males[["age","region"]], drop_first=True)

# Adding the results to the master dataframe
males = pd.concat([males, dummy_males], axis=1)

In [ ]:
males.head()

In [ ]:
males.drop(columns=["age","region"], inplace=True)

In [ ]:
males.head()

In [ ]:
dummy_females = pd.get_dummies(females[["age","region"]], drop_first=True)

# Adding the results to the master dataframe
females = pd.concat([females, dummy_females], axis=1)


In [ ]:
females.head()

In [ ]:
females.drop(columns=["age","region"], inplace=True)
females.head()

### Test-Train split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
np.random.seed(0)
df_train,df_test= train_test_split(males, train_size=0.7, random_state=100)


In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()

In [ ]:
df_train[["charges","bmi"]]=scaler.fit_transform(df_train[["charges","bmi"]])

In [ ]:
y_train=df_train[["charges"]]
X_train=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_northwest","region_southeast","region_southwest"]]

In [ ]:
df_test[["charges","bmi"]]=scaler.transform(df_test[["charges","bmi"]])

In [ ]:
y_test=df_train[["charges"]]
X_test=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_northwest","region_southeast","region_southwest"]]

## We will build Decision Tree, Multi-Linear Regression, Cross-validation on decision trees and random forests to check the predictive power of each models and therefore compare them accordingly



### Building a Decision Tree for predicting "charges" for males


In [ ]:
from sklearn.tree import DecisionTreeRegressor


In [ ]:
dt=DecisionTreeRegressor(random_state=42, max_depth=3, min_samples_leaf=20)

In [ ]:
X_train.head()

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
y_test_pred=dt.predict(X_test)

In [ ]:
y_test_pred[:10]

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_test_pred)

### Building a Linear Model for predicting "charges" for males

In [ ]:
import statsmodels.api as sm

In [ ]:
y_train=df_train[["charges"]]
X_train=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_northwest","region_southeast","region_southwest"]]
y_test=df_test[["charges"]]
X_test=df_test[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_northwest","region_southeast","region_southwest"]]

In [ ]:
X_train_lm=sm.add_constant(X_train)

In [ ]:
lr=sm.OLS(y_train,X_train_lm).fit()

In [ ]:
lr.params

In [ ]:
print(lr.summary()) ## We will have to drop "region_northwest" as the p-value lies in critical zone and needs to be rejected

In [ ]:
X_train=X_train.drop(columns=["region_northwest"])
X_train_lm=sm.add_constant(X_train)
lr=sm.OLS(y_train,X_train_lm).fit()
print(lr.summary())                 ## We will have to drop "region_southeast" as the p-value lies in critical zone and needs to be rejected

In [ ]:
X_train=X_train.drop(columns=["region_southeast"])
X_train_lm=sm.add_constant(X_train)
lr=sm.OLS(y_train,X_train_lm).fit()
print(lr.summary())      ## We will have to drop "region_southwest" as the p-value lies in critical zone and needs to be rejected

In [ ]:
X_train=X_train.drop(columns=["region_southwest"])
X_train_lm=sm.add_constant(X_train)
lr=sm.OLS(y_train,X_train_lm).fit()
print(lr.summary())   ## We will have to drop "children" as the p-value lies in critical zone and needs to be rejected

In [ ]:
X_train=X_train.drop(columns=["children"])
X_train_lm=sm.add_constant(X_train)
lr=sm.OLS(y_train,X_train_lm).fit()
print(lr.summary())    

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_test=X_test[X_train.columns]
X_test_lm=sm.add_constant(X_test)

y_test_pred=lr.predict(X_test_lm)

In [ ]:
y_test_pred[:5]

In [ ]:
r2_score(y_test,y_test_pred)

In [ ]:
y_test[:5]

### Cross-Validation using GridSearch CV for predicting "charges" for males

In [ ]:
from sklearn.model_selection import GridSearchCV
dt=DecisionTreeRegressor()

In [ ]:
params={"max_depth":[2,3,4],"min_samples_leaf":[5,10,15,20,25]}

In [ ]:
grid_search=GridSearchCV(estimator=dt, param_grid=params,cv=4,n_jobs=-1,verbose=1, scoring="r2")


In [ ]:
y_train=df_train[["charges"]]
X_train=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_northwest","region_southeast","region_southwest"]]

In [ ]:
%%time
grid_search.fit(X_train,y_train)

In [ ]:
grid_search.best_score_

In [ ]:
dt_best=grid_search.best_estimator_
dt_best

In [ ]:
y_test=df_train[["charges"]]
X_test=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_northwest","region_southeast","region_southwest"]]
y_test_pred=dt_best.predict(X_test)


In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_test_pred)

### Random Forests Regressor for predicting "charges" for males

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf=RandomForestRegressor(random_state=42, n_jobs=-1,max_depth=5)

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
rf.feature_importances_

In [ ]:
imp_df=pd.DataFrame({"var_name":X_train.columns,"Importance":rf.feature_importances_})
imp_df.sort_values(by="Importance", ascending=False)

In [ ]:
y_test_pred=rf.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_test_pred)

## Concluding R2_SCORES for males


In [ ]:
scores=[78.64, 87.18, 88.35, 90.86]
df_males=pd.DataFrame({"model":["Linear Regression","Decision Tree","cross validation using decision trees","random forests"],"r2_scores":scores})

In [ ]:
df_males

### Building a Decision Tree for predicting "charges" for females

In [ ]:
np.random.seed(0)
df_train,df_test= train_test_split(females, train_size=0.7, random_state=100)

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [ ]:
df_train[["charges","bmi"]]=scaler.fit_transform(df_train[["charges","bmi"]])

In [ ]:
y_train=df_train[["charges"]]
X_train=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_northwest","region_southeast","region_southwest"]]

In [ ]:
df_test[["charges","bmi"]]=scaler.transform(df_test[["charges","bmi"]])

In [ ]:
y_test=df_train[["charges"]]
X_test=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_northwest","region_southeast","region_southwest"]]

In [ ]:
dt=DecisionTreeRegressor(random_state=42, max_depth=3, min_samples_leaf=15)
dt.fit(X_train, y_train)

In [ ]:
y_test_pred=dt.predict(X_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_test_pred)

### Building a Linear Model for predicting "charges" for females

In [ ]:
X_train=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_northwest","region_southeast","region_southwest"]]
y_train=df_train[["charges"]]
                 
    

In [ ]:

X_train_lm=sm.add_constant(X_train)
lr=sm.OLS(y_train,X_train_lm).fit()
print(lr.summary())   ## We will have to drop "region_northwest" as the p-value lies in critical zone and needs to be rejected

In [ ]:
X_train=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_southeast","region_southwest"]]
X_train_lm=sm.add_constant(X_train)
lr=sm.OLS(y_train,X_train_lm).fit()
print(lr.summary())    ## We will have to drop "region_southwest" as the p-value lies in critical zone and needs to be rejected

In [ ]:
X_train=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_southeast"]]
X_train_lm=sm.add_constant(X_train)
lr=sm.OLS(y_train,X_train_lm).fit()
print(lr.summary())     ## We will have to drop "region_southeast" as the p-value lies in critical zone and needs to be rejected

In [ ]:
X_train=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood"]]
X_train_lm=sm.add_constant(X_train)
lr=sm.OLS(y_train,X_train_lm).fit()
print(lr.summary())

In [ ]:
X_test=df_test[X_train.columns]
X_test_lm=sm.add_constant(X_test)
y_test_pred=lr.predict(X_test_lm)
y_test=df_test[["charges"]]

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_test_pred)

In [ ]:
X_train=df_train[["bmi","children","smoker","age_older_adulthood","age_young_adulthood","region_northwest","region_southeast","region_southwest"]]
y_train=df_train[["charges"]]
dt=DecisionTreeRegressor()
grid_search=GridSearchCV(estimator=dt,param_grid=params,cv=4,n_jobs=-1,verbose=1,scoring="r2")

grid_search.fit(X_train,y_train)



In [ ]:
grid_search.best_score_

In [ ]:
dt_best=grid_search.best_estimator_

In [ ]:
X_test=df_test[X_train.columns]
y_test=df_test[["charges"]]
y_test_pred=dt_best.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_test_pred)

### Random Forest Regressor for predicting "charges" for females

In [ ]:
rf=RandomForestRegressor(random_state=42, n_jobs=-1,max_depth=5,n_estimators=100)
rf.fit(X_train,y_train)

In [ ]:
rf.feature_importances_

In [ ]:
imp_df=pd.DataFrame({"var_name":X_train.columns,"Importance":rf.feature_importances_})
imp_df.sort_values(by="Importance", ascending=False)

In [ ]:
y_test_pred=rf.predict(X_test)

In [ ]:
r2_score(y_test, y_test_pred)

In [ ]:
scores=[62.89, 83.41, 75.35, 75.04]
df_females=pd.DataFrame({"model":["Linear Regression","Decision Tree","cross validation using decision trees","random forests"],"r2_scores":scores})

In [ ]:
df_females

In [ ]:
prediction_scores=pd.merge(df_males,df_females, on="model")
prediction_scores

In [ ]:
prediction_scores=prediction_scores.rename(columns={"r2_scores_x":"r2_scores_males","r2_scores_y":"r2_scores_females"})

In [ ]:
prediction_scores.set_index('model')

#### Hence, Multi-Linear Regression model predicting charges for males is :  0.0372 + 0.21(bmi) + 0.4(smoker) + 0.06(older_age) - 0.06(younger)


#### Multi-Linear Regression model predicting charges for females is :  0.006 + 0.17(bmi) + 0.001(children) + 0.37(smoker) + 0.09(older_age) - 0.03(younger)


### Using Random Forests, we could see that the top 5 feature importance(left to right in decreasing degree of importance) for predicting charges for males and females are:

| Gender | Feature1 | Feature2 | Feature3 | Feature4 | Feature5 |
| --- | --- | --- | --- | --- | --- |
| Male | smoker | bmi | older age(>45) | younger age(18-35) | children|
| Female | smoker | bmi | older age(>45) | children | younger age(18-35) | 


### younger age has negative corelation with "charges" which is expected.